In [1]:
# --- (Ensure metadata_dfs, all_layer_assignments, all_layer_factors, dates, device_labels are loaded) ---

def analyze_active_interactions_for_layer(layer_name_to_analyze, active_interactions_df,
                                        community_assignments_for_layer_N_T, # N x T matrix
                                        factor_C_for_layer_T_R, # T x R matrix
                                        device_labels_list, dates_list):
    print(f"\n--- Correlating Active Interactions with Layer: {layer_name_to_analyze} ---")
    if community_assignments_for_layer_N_T is None:
        print(f"  No community assignments found for {layer_name_to_analyze}")
        return

    for _, event in active_interactions_df.iterrows():
        device_name = event['Device']
        event_date_str = event['Date'].strftime('%Y-%m-%d')
        # Note: Event times are specific, daily matrices are coarse.
        # We look at the device's cluster on the DAY of interaction.

        if device_name not in device_labels_list:
            # print(f"  Skipping event for device '{device_name}' (not in known device list or name mismatch)")
            continue
        
        device_idx = device_labels_list.index(device_name)

        if event_date_str in dates_list:
            time_idx = dates_list.index(event_date_str)
            assigned_cluster = community_assignments_for_layer_N_T[device_idx, time_idx] + 1 # 1-based
            
            print(f"  Device: {device_name}, Event Date: {event_date_str}")
            print(f"    Assigned to {layer_name_to_analyze} Cluster {assigned_cluster} on this day.")

            # Check Factor C activity for that cluster on that day
            if factor_C_for_layer_T_R is not None and assigned_cluster-1 < factor_C_for_layer_T_R.shape[1]:
                cluster_activity_on_event_day = factor_C_for_layer_T_R[time_idx, assigned_cluster-1]
                print(f"    Temporal activity of assigned Cluster {assigned_cluster} on this day: {cluster_activity_on_event_day:.2f}")
            
            # TODO for deeper analysis:
            # - Look at cluster assignment *before* and *after* the event day.
            # - If interaction is short, daily aggregation hides it. This analysis shows general state.
            # - Could try to look at raw matrix values if finer granularity is needed for short events.
        else:
            # print(f"  Skipping event for {device_name} on {event_date_str} (date not in processed range)")
            pass
    print("-" * 30)


def analyze_passive_interactions_for_layer(layer_name_to_analyze, passive_interactions_df,
                                           community_assignments_for_layer_N_T,
                                           factor_C_for_layer_T_R,
                                           device_labels_list, dates_list,
                                           sensor_device_names): # List of names like cameras, motion sensors
    print(f"\n--- Correlating Passive Interactions with Layer: {layer_name_to_analyze} ---")
    if community_assignments_for_layer_N_T is None or factor_C_for_layer_T_R is None:
        print(f"  No community assignments or Factor C found for {layer_name_to_analyze}")
        return

    for _, event_period in passive_interactions_df.iterrows():
        period_date_str = event_period['Date'].strftime('%Y-%m-%d')
        print(f"  Passive Interaction Period: {period_date_str} (Time: {event_period['Start Time']} - {event_period['End Time']})")

        if period_date_str in dates_list:
            time_idx = dates_list.index(period_date_str)
            
            for device_name in sensor_device_names:
                if device_name not in device_labels_list: continue
                device_idx = device_labels_list.index(device_name)
                
                assigned_cluster = community_assignments_for_layer_N_T[device_idx, time_idx] + 1
                cluster_activity_on_event_day = factor_C_for_layer_T_R[time_idx, assigned_cluster-1]
                
                print(f"    Device: {device_name}, Assigned Cluster: {assigned_cluster}, Cluster Activity: {cluster_activity_on_event_day:.2f}")
        else:
            # print(f"  Skipping passive period on {period_date_str} (date not in processed range)")
            pass
    print("-" * 30)

# --- In your Jupyter Notebook, after loading all_layer_assignments and all_layer_factors ---

# Example usage:
target_layer = 'external_tcp_tls_count' # Choose a layer to analyze
if target_layer in all_layer_assignments:
    assignments = all_layer_assignments[target_layer]
    factors = all_layer_factors[target_layer]
    
    analyze_active_interactions_for_layer(target_layer, active_interactions_df, assignments, factors['C'], device_labels, dates)
    
    # Define which of your devices are "sensor-like" for passive analysis
    sensor_devices = ["Netatmo Smart Indoor Security Camera", "Perfk Motion Sensor", "SAMSUNG Pan/Tilt 1080P Wi-Fi Camera", "EZVIZ Security Camera"] # Example list
    analyze_passive_interactions_for_layer(target_layer, passive_interactions_df, assignments, factors['C'], device_labels, dates, sensor_devices)

# Repeat for other layers of interest

NameError: name 'all_layer_assignments' is not defined